In [1]:
import pandas as pd
import json
import os
from datetime import date

# Collecting the data from Twitter

In [2]:
# Import the Twython class
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])


In [3]:
import pysad

In [12]:
import importlib
importlib.reload(pysad)

<module 'pysad' from '/home/benjamin/Documents/EPFL/Research/sad/sad_tweets/pysad.py'>

In [42]:
init_accounts = pysad.initial_accounts()

In [43]:
init_accounts.list()

['swiss_climate_regular',
 'swiss_climate_controversial',
 'swiss_immigration',
 'french_tech_lesechos']

In [44]:

    
######Choose a category##############    
#category_name = 'swiss_climate_controversial'
#category_name = 'swiss_climate_regular'
#category_name = 'french_tech_lesechos'
category_name = 'swiss_immigration'
#####################################

username_list = init_accounts.accounts(category_name)

# create the path to save the experiment indexed with a date
today = date.today()
date_string = today.strftime("%Y%m%d")
print("date string =", date_string)

#date_string = '191128'

tweet_data_path_list = ['tweetdata', category_name, date_string]
results_data_path_list = ['resultsdata', category_name, date_string]
#get_tweets = python_tweets.get_user_timeline(screen_name = username,  
#         count = 200, include_rts = True)

def initialize_folder(path_folder_list):
    folder_concat = ''
    for folder in path_folder_list[:-1]:
        folder_concat += folder + '/'
        if not os.path.isdir(folder_concat):
            os.mkdir(folder_concat)
            print('Path created:',folder_concat)
    # Special treatment for the last folder
    folder_concat += path_folder_list[-1] + '/'
    if not os.path.isdir(folder_concat):
        os.mkdir(folder_concat)
        print('Path created:',folder_concat)
    else:
        for f in os.listdir(folder_concat):
            os.remove(os.path.join(folder_concat, f))
        print('Cleaned path',folder_concat)
    return folder_concat

tweet_data_path = initialize_folder(tweet_data_path_list)
results_data_path = initialize_folder(results_data_path_list)

date string = 20200129
Path created: tweetdata/swiss_immigration/
Path created: tweetdata/swiss_immigration/20200129/
Path created: resultsdata/swiss_immigration/
Path created: resultsdata/swiss_immigration/20200129/


In [45]:
username_list

['Kalvingrad1291',
 'democratesuisse',
 'VigilanceIslam',
 'lioneljonson01',
 'ChWilhou',
 'HunterSThomson2',
 'A_Addams_',
 'ObservateursCH',
 'JuanCandida',
 'novopress']

In [46]:
min_mentions = 2 # minimal number of mentions of a user to be followed
max_day_old = 7 # number max of days in the past
exploration_depth = 2 # mention of mention of mention of ... up to exploration depth

In [47]:
total_user_list = pysad.collect_tweets(username_list, tweet_data_path, python_tweets, min_mentions=min_mentions,
               max_day_old=max_day_old, exploration_depth=exploration_depth)

  0%|          | 0/10 [00:00<?, ?it/s]

Threshold set to 2 mentions.

******* Processing users at 0-hop distance *******


 40%|████      | 4/10 [00:02<00:04,  1.35it/s]

Twitter API returned error 404 for user lioneljonson01.


  0%|          | 0/21 [00:00<?, ?it/s]

users with empty tweet list or no mention: ['Kalvingrad1291', 'democratesuisse', 'lioneljonson01', 'ObservateursCH', 'JuanCandida', 'novopress']

******* Processing users at 1-hop distance *******


100%|██████████| 21/21 [00:17<00:00,  1.17it/s]

users with empty tweet list or no mention: []
Total number of users collected:
194 194


In [48]:
len(total_user_list)

194

# Loading the saved data into an edge table

In [49]:
import glob

edge_df = pd.DataFrame()
for filename in glob.glob(tweet_data_path + '*_mentions' +'_t' +str(min_mentions)+ '.json'):
    new_edge_df = pd.read_json(filename)
    print('{} with {} tweets.'.format(filename,len(new_edge_df)))
    edge_df = edge_df.append(new_edge_df)
edge_df.reset_index(drop=True, inplace=True)

tweetdata/swiss_immigration/20200129/SOSMedFrance_mentions_t2.json with 3 tweets.
tweetdata/swiss_immigration/20200129/SouSonOeil_mentions_t2.json with 42 tweets.
tweetdata/swiss_immigration/20200129/docteurpepe_mentions_t2.json with 69 tweets.
tweetdata/swiss_immigration/20200129/Lulu_skyraptor_mentions_t2.json with 44 tweets.
tweetdata/swiss_immigration/20200129/SOS_Racisme_mentions_t2.json with 14 tweets.
tweetdata/swiss_immigration/20200129/ChWilhou_mentions_t2.json with 62 tweets.
tweetdata/swiss_immigration/20200129/MarleneSchiappa_mentions_t2.json with 52 tweets.
tweetdata/swiss_immigration/20200129/Nguyen11804422_mentions_t2.json with 74 tweets.
tweetdata/swiss_immigration/20200129/CretinusAlp_mentions_t2.json with 61 tweets.
tweetdata/swiss_immigration/20200129/Bellica_fr_mentions_t2.json with 14 tweets.
tweetdata/swiss_immigration/20200129/IceWind86585905_mentions_t2.json with 78 tweets.
tweetdata/swiss_immigration/20200129/A_Addams__mentions_t2.json with 42 tweets.
tweetdata

In [50]:
DEGREE_MIN = 2 # Minimal number of connections in the graph

G = pysad.graph_from_edgeslist(edge_df,DEGREE_MIN)
G.name = category_name
G.date = date_string

Creating the graph from the edge list
Field "hashtags" of class <class 'list'> converted to json string
Field "date" of class <class 'list'> converted to json string
Field "urls" of class <class 'list'> converted to json string
Field "text" of class <class 'list'> converted to json string
Nb of nodes: 1038
Nb of nodes after removing nodes with degree strictly smaller than 2: 191
removed 0 isolated nodes.


## Community detection and save full graph

In [51]:
G,clusters = pysad.detect_communities(G)

Communities saved on the graph as node attributes.
Nb of partitions: 5


In [52]:
# Save the graph
import networkx as nx

graphname = 'globalgraph'

graphfilename = results_data_path + graphname + '_t' + str(min_mentions) + '_md' + str(DEGREE_MIN) +'_graph.gexf'
nx.write_gexf(G,graphfilename)
#nx.write_yaml(G,graphfilename)
#nx.write_pajek(G,graphfilename)
#nx.node_link_data(G,graphfilename)
print('Wrote',graphfilename)

Wrote resultsdata/swiss_immigration/20200129/globalgraph_t2_md2_graph.gexf


## Analyze clusters

## Automatic processing of all clusters

In [53]:
for c_id in clusters:
    cgraph = clusters[c_id]
    cgraph = pysad.cluster_attributes(cgraph)
    table_dic = pysad.cluster_tables(cgraph)
    cluster_filename = results_data_path + 'cluster' + str(c_id)
    pysad.save_excel(table_dic,cluster_filename + '_infos.xlsx')
    pysad.save_graph(cgraph,cluster_filename + 'graph.gexf')

Data saved to resultsdata/swiss_immigration/20200129/cluster0_infos.xlsx
Graph saved to resultsdata/swiss_immigration/20200129/cluster0graph.gexf
Data saved to resultsdata/swiss_immigration/20200129/cluster1_infos.xlsx
Graph saved to resultsdata/swiss_immigration/20200129/cluster1graph.gexf
Data saved to resultsdata/swiss_immigration/20200129/cluster2_infos.xlsx
Graph saved to resultsdata/swiss_immigration/20200129/cluster2graph.gexf
Data saved to resultsdata/swiss_immigration/20200129/cluster3_infos.xlsx
Graph saved to resultsdata/swiss_immigration/20200129/cluster3graph.gexf
Data saved to resultsdata/swiss_immigration/20200129/cluster4_infos.xlsx
Graph saved to resultsdata/swiss_immigration/20200129/cluster4graph.gexf
